
https://tianchi.aliyun.com/competition/entrance/231593/introduction
官方網站

https://redstonewill.com/1681/
紅色石頭: 介紹與範例

https://github.com/RedstoneWill/MachineLearningInAction-Camp/blob/master/Week1/o2o-1.ipynb
(0.532344469452)

https://github.com/RedstoneWill/MachineLearningInAction-Camp/blob/master/Week5/Assignments/o2o-1.ipynb
(0.532344469452)

https://github.com/RedstoneWill/MachineLearningInAction-Camp/blob/master/Week10/Assignments/o2o-2.ipynb
(0.630400363653)

https://github.com/wepe/O2O-Coupon-Usage-Forecast
第一名隊伍的註解

https://www.kaggle.com/lianglirong/alibaba-tianchi-o2o
(0.9094526220772331)
範例

https://tianchi.aliyun.com/notebook-ai/detail?spm=5176.12586969.1002.15.29281b48IrTtER&postId=8462
(0.909452622077)
100行代码入门天池O2O优惠券使用新人赛【精简教程版】

https://www.kaggle.com/cbrogan/xgboost-example-python
XGBoost example (Python)


In [0]:
#https://www.kaggle.com/ml100marathon/baseline-example

In [3]:
from google.colab import drive
drive.mount('content/')
!ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at content/
content  sample_data


In [4]:
import os
os.chdir("/content/content/My Drive/DL_2019/2nd-ML100Days/Port40D51-D53")
!ls

baseline_example1.csv		ml100marathon-02-01.zip
column_description.csv		sample_submission.csv
column_description_en.csv	test_offline.csv
Day_051_baseline-example.ipynb	train_offline.csv
Day_051_HW.ipynb


In [0]:
import os
import numpy as np
import pandas as pd
from datetime import date

from sklearn.model_selection import KFold, train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, roc_auc_score, auc, roc_curve
from sklearn.preprocessing import MinMaxScaler

DATA_ROOT = "./"

In [6]:
dfoff = pd.read_csv(os.path.join(DATA_ROOT,'train_offline.csv'))
dftest = pd.read_csv(os.path.join(DATA_ROOT,'test_offline.csv'))
dftest = dftest[~dftest.Coupon_id.isna()]
dftest.reset_index(drop=True, inplace=True)
print('train data: ', dfoff.shape)
print('test data: ', dftest.shape)

train data:  (1160742, 7)
test data:  (306313, 6)


In [7]:
dfoff.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN


In [8]:
dftest.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received
0,1439408,4663,11002.0,150:20,1.0,20160528.0
1,1439408,2632,8591.0,20:1,0.0,20160613.0
2,1439408,2632,8591.0,20:1,0.0,20160516.0
3,2029232,450,1532.0,30:5,0.0,20160530.0
4,2029232,6459,12737.0,20:1,0.0,20160519.0


In [9]:
#資料量太大了，所以開發時先用較少量的資料來玩，調整差不多後，再用完整資料來玩
#dfoff = dfoff[:100000]
#dftest = dftest[:10000]

print(dfoff.shape)
print(dftest.shape)

(1160742, 7)
(306313, 6)


In [10]:
## Creat target label 
"""
According to the definition, 
1) buy with coupon within (include) 15 days ==> 1
2) buy with coupon but out of 15 days ==> 0
3) buy without coupon ==> -1 (we don't care)
"""
def label(row):
    if np.isnan(row['Date_received']):
        return -1
    if not np.isnan(row['Date']):
        td = pd.to_datetime(row['Date'], format='%Y%m%d') -  pd.to_datetime(row['Date_received'], format='%Y%m%d')
        if td <= pd.Timedelta(15, 'D'):
            return 1
    return 0

dfoff["label"] = dfoff.apply(label, axis=1)
dfoff["label"].value_counts()

 0    710665
-1    413773
 1     36304
Name: label, dtype: int64

In [11]:
dfoff.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0,-1
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,0
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN,0
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN,0
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN,0


In [0]:
# Generate features - weekday acquired coupon
def getWeekday(row):
    if (np.isnan(row)) or (row==-1):
        return row
    else:
        return pd.to_datetime(row, format = "%Y%m%d").dayofweek+1 # add one to make it from 0~6 -> 1~7

dfoff['weekday'] = dfoff['Date_received'].apply(getWeekday)
dftest['weekday'] = dftest['Date_received'].apply(getWeekday)

# weekday_type (weekend = 1)
dfoff['weekday_type'] = dfoff['weekday'].astype('str').apply(lambda x : 1 if x in [6,7] else 0 ) # apply to trainset
dftest['weekday_type'] = dftest['weekday'].astype('str').apply(lambda x : 1 if x in [6,7] else 0 ) # apply to testset

In [13]:
dfoff.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label,weekday,weekday_type
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0,-1,NaN,0
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,0,3.0,0
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN,0,6.0,0
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN,0,5.0,0
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN,0,5.0,0


In [14]:
weekdaycols = ['weekday_' + str(i) for i in range(1,8)]
print(weekdaycols)

tmpdf = pd.get_dummies(dfoff['weekday'].replace(-1, np.nan))
tmpdf.columns = weekdaycols
dfoff[weekdaycols] = tmpdf

tmpdf = pd.get_dummies(dftest['weekday'].replace(-1, np.nan))
tmpdf.columns = weekdaycols
dftest[weekdaycols] = tmpdf

['weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']


In [15]:
dfoff.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label,weekday,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0,-1,NaN,0,0,0,0,0,0,0,0
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,0,3.0,0,0,0,1,0,0,0,0
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN,0,6.0,0,0,0,0,0,0,1,0
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN,0,5.0,0,0,0,0,0,1,0,0
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN,0,5.0,0,0,0,0,0,1,0,0


In [0]:
# Generate features - coupon discount and distance
def getDiscountType(row):
    if row == 'null':
        return 'null'
    elif ':' in row:
        return 1
    else:
        return 0

def convertRate(row):
    """Convert discount to rate"""
    if row == 'null':
        return 1.0
    elif ':' in row:
        rows = row.split(':')
        return 1.0 - float(rows[1])/float(rows[0])
    else:
        return float(row)

def getDiscountMan(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[0])
    else:
        return 0

def getDiscountJian(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[1])
    else:
        return 0

def processData(df):
    
    # convert discunt_rate
    df['discount_rate'] = df['Discount_rate'].astype('str').apply(convertRate)
    df['discount_man'] = df['Discount_rate'].astype('str').apply(getDiscountMan)
    df['discount_jian'] = df['Discount_rate'].astype('str').apply(getDiscountJian)
    df['discount_type'] = df['Discount_rate'].astype('str').apply(getDiscountType)
    
    # convert distance
    df.loc[df.Distance.isna(), "Distance"] = 99
    return df

dfoff = processData(dfoff)
dftest = processData(dftest)

In [17]:
print('折扣種類: ', dfoff['discount_rate'].unique())

折扣種類:  [       nan 0.95       0.9        0.5        0.85       0.75
 0.83333333 0.8        0.93333333 0.7        0.6        0.86666667
 0.66666667 0.96666667 0.98       0.99       0.975      0.33333333
 0.2        0.4       ]


In [18]:
print('距離種類: ', dfoff['Distance'].unique())

距離種類:  [ 0.  1.  2. 99. 10.  4.  7.  9.  3.  5.  6.  8.]


In [19]:
dfoff.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label,weekday,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,discount_rate,discount_man,discount_jian,discount_type
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0,-1,NaN,0,0,0,0,0,0,0,0,NaN,0,0,0
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,0,3.0,0,0,0,1,0,0,0,0,0.95,20,1,1
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN,0,6.0,0,0,0,0,0,0,1,0,0.95,20,1,1
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN,0,5.0,0,0,0,0,0,1,0,0,0.90,200,20,1
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN,0,5.0,0,0,0,0,0,1,0,0,0.90,200,20,1


# 訓練用的 feature

In [20]:
original_feature = ['discount_rate',
                    'discount_type',
                    'discount_man', 
                    'discount_jian',
                    'Distance', 
                    'weekday', 
                    'weekday_type'] + weekdaycols
print(len(original_feature),original_feature)

predictors = original_feature
print(predictors)

14 ['discount_rate', 'discount_type', 'discount_man', 'discount_jian', 'Distance', 'weekday', 'weekday_type', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']
['discount_rate', 'discount_type', 'discount_man', 'discount_jian', 'Distance', 'weekday', 'weekday_type', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']


# 切 訓練&驗證 用的資料

In [21]:
## Naive model
def split_train_valid(row, date_cut="20160416"):
    is_train = True if pd.to_datetime(row, format="%Y%m%d") < pd.to_datetime(date_cut, format="%Y%m%d") else False
    return is_train
    
df = dfoff[dfoff['label'] != -1].copy()
df["is_train"] = df["Date_received"].apply(split_train_valid)
train = df[df["is_train"]]
valid = df[~df["is_train"]]
train.reset_index(drop=True, inplace=True)
valid.reset_index(drop=True, inplace=True)

print("Train size: {}, #positive: {}".format(len(train), train["label"].sum()))
print("Valid size: {}, #positive: {}".format(len(valid), valid["label"].sum()))

Train size: 667753, #positive: 32472
Valid size: 79216, #positive: 3832


# 建模型

In [22]:
def check_model(data, predictors):
    
    classifier = lambda: SGDClassifier(
        loss='log', 
        penalty='elasticnet', 
        fit_intercept=True, 
        max_iter=100, 
        shuffle=True, 
        n_jobs=1,
        class_weight=None)

    model = Pipeline(steps=[
        ('ss', StandardScaler()),
        ('en', classifier())
    ])

    parameters = {
        'en__alpha': [ 0.001, 0.01, 0.1],
        'en__l1_ratio': [ 0.001, 0.01, 0.1]
    }

    folder = StratifiedKFold(n_splits=3, shuffle=True)
    
    grid_search = GridSearchCV(
        model, 
        parameters, 
        cv=folder, 
        n_jobs=-1, 
        verbose=1)
    grid_search = grid_search.fit(data[predictors], 
                                  data['label'])
    
    return grid_search

model = check_model(train, predictors)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:   33.0s finished


# 驗證預測

In [0]:
y_valid_pred = model.predict_proba(valid[predictors])
valid1 = valid.copy()
valid1['pred_prob'] = y_valid_pred[:, 1]

In [24]:
valid1.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label,weekday,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,discount_rate,discount_man,discount_jian,discount_type,is_train,pred_prob
0,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN,0,5.0,0,0,0,0,0,1,0,0,0.900000,200,20,1,False,0.006797
1,163606,1569,5054.0,200:30,10.0,20160421.0,NaN,0,4.0,0,0,0,0,1,0,0,0,0.850000,200,30,1,False,0.006133
2,4061024,3381,7610.0,200:20,10.0,20160426.0,NaN,0,2.0,0,0,1,0,0,0,0,0,0.900000,200,20,1,False,0.009304
3,106443,450,3732.0,30:5,99.0,20160429.0,NaN,0,5.0,0,0,0,0,0,1,0,0,0.833333,30,5,1,False,0.057884
4,114747,1569,5054.0,200:30,9.0,20160426.0,NaN,0,2.0,0,0,1,0,0,0,0,0,0.850000,200,30,1,False,0.006253


# 評分

In [25]:
from sklearn.metrics import roc_auc_score, accuracy_score

auc_score = roc_auc_score(y_true=valid.label, y_score=y_valid_pred[:,1])
acc = accuracy_score(y_true=valid.label, y_pred=y_valid_pred.argmax(axis=1))

print("Validation AUC: {:.3f}, Accuracy: {:.3f}".format(auc_score, acc))

Validation AUC: 0.746, Accuracy: 0.952


# 輸出預測檔案

In [0]:
def exportSubmit(model, filename):

    targetset = dftest.copy()
    print(targetset.shape)

    targetset = targetset[~targetset.Coupon_id.isna()]
    targetset.reset_index(drop=True, inplace=True)
    testset = targetset[predictors].copy()

    y_test_pred = model.predict_proba(testset[predictors])
    test1 = testset.copy()
    test1['pred_prob'] = y_test_pred[:, 1]
    print(test1.shape)
    
    output = pd.concat((targetset[["User_id", "Coupon_id", "Date_received"]], test1["pred_prob"]), axis=1)
    print(output.shape)

    output.loc[:, "User_id"] = output["User_id"].apply(lambda x:str(int(x)))
    output.loc[:, "Coupon_id"] = output["Coupon_id"].apply(lambda x:str(int(x)))
    output.loc[:, "Date_received"] = output["Date_received"].apply(lambda x:str(int(x)))
    output["uid"] = output[["User_id", "Coupon_id", "Date_received"]].apply(lambda x: '_'.join(x.values), axis=1)
    output.reset_index(drop=True, inplace=True)
    
    ### NOTE: YOUR SUBMITION FILE SHOULD HAVE COLUMN NAME: uid, label
    out = output.groupby("uid", as_index=False).mean()
    out = out[["uid", "pred_prob"]]
    out.columns = ["uid", "label"]
    out.to_csv(filename, header=["uid", "label"], index=False) # submission format
    out.head()

In [27]:
exportSubmit(model, "Day_051_HW_baseline1.csv")
#score:0.58841

(306313, 19)
(306313, 15)
(306313, 4)


# 使用 LogisticRegression

In [0]:
# 使用三種模型 : 邏輯斯迴歸 / 梯度提升機 / 隨機森林, 參數使用 Random Search 尋找
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

In [29]:
# 線性迴歸預測檔 (結果有部分隨機, 請以 Kaggle 計算的得分為準, 以下模型同理)
lr = LogisticRegression(tol=0.001, penalty='l2', fit_intercept=True, C=1.0)

lr.fit(train[predictors], train['label'])
lr_pred = lr.predict_proba(valid[predictors])

auc_score = roc_auc_score(y_true=valid.label, y_score=lr_pred[:,1])
acc = accuracy_score(y_true=valid.label, y_pred=lr_pred.argmax(axis=1))

print("Validation AUC: {:.3f}, Accuracy: {:.3f}".format(auc_score, acc))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Validation AUC: 0.749, Accuracy: 0.952


In [30]:
exportSubmit(lr, "Day_051_HW_lr1.csv")
#0.59369

(306313, 19)
(306313, 15)
(306313, 4)


# 使用 GradientBoostingClassifier

In [31]:
# 梯度提升機預測檔 
gdbt = GradientBoostingClassifier(tol=100, subsample=0.75, n_estimators=250, max_features=len(predictors),
                                  max_depth=6, learning_rate=0.03)

gdbt.fit(train[predictors], train['label'])
gdbt_pred = gdbt.predict_proba(valid[predictors])

auc_score = roc_auc_score(y_true=valid.label, y_score=gdbt_pred[:,1])
acc = accuracy_score(y_true=valid.label, y_pred=gdbt_pred.argmax(axis=1))

print("Validation AUC: {:.3f}, Accuracy: {:.3f}".format(auc_score, acc))

exportSubmit(gdbt, "Day_051_HW_gdbt1.csv")
#0.62440

Validation AUC: 0.788, Accuracy: 0.951
(306313, 19)
(306313, 15)
(306313, 4)


# 使用 RandomForestClassifier

In [32]:
# 隨機森林預測檔
rf = RandomForestClassifier(n_estimators=100, min_samples_split=2, min_samples_leaf=1, 
                            max_features='sqrt', max_depth=6, bootstrap=True)

rf.fit(train[predictors], train['label'])
rf_pred = rf.predict_proba(valid[predictors])

auc_score = roc_auc_score(y_true=valid.label, y_score=rf_pred[:,1])
acc = accuracy_score(y_true=valid.label, y_pred=rf_pred.argmax(axis=1))

print("Validation AUC: {:.3f}, Accuracy: {:.3f}".format(auc_score, acc))

exportSubmit(rf, "Day_051_HW_rf1.csv")
#0.63762

Validation AUC: 0.789, Accuracy: 0.952
(306313, 19)
(306313, 15)
(306313, 4)


# 使用 xgboost

In [0]:
import xgboost as xgb

In [0]:
gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(train[predictors], train['label'])
gbm_pred = gbm.predict(valid[predictors])

In [35]:
auc_score = roc_auc_score(y_true=valid.label, y_score=gbm_pred)
acc = accuracy_score(y_true=valid.label, y_pred=gbm_pred)

print("Validation AUC: {:.3f}, Accuracy: {:.3f}".format(auc_score, acc))

Validation AUC: 0.500, Accuracy: 0.952


In [36]:
exportSubmit(gbm, "Day_051_HW_xgboost1.csv")
#0.63762

(306313, 19)
(306313, 15)
(306313, 4)


# **建模型**

In [38]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score

# 建立模型 (使用 20 顆樹，每棵樹的最大深度為 4)
clf = RandomForestClassifier(n_estimators=20, max_depth=4)

# 訓練模型
clf.fit(train[predictors], train['label'])

# 預測測試集
y_pred = clf.predict(valid[predictors])

# 評分
auc_score = roc_auc_score(y_true=valid.label, y_score=y_pred)
acc = accuracy_score(y_true=valid.label, y_pred=y_pred)

print("Validation AUC: {:.3f}, Accuracy: {:.3f}".format(auc_score, acc))

Validation AUC: 0.500, Accuracy: 0.952


In [39]:
# 比較重要的 feature
print(len(predictors), predictors)
print("Feature importance: ", len(clf.feature_importances_), clf.feature_importances_)

14 ['discount_rate', 'discount_type', 'discount_man', 'discount_jian', 'Distance', 'weekday', 'weekday_type', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']
Feature importance:  14 [9.50620727e-02 1.26555292e-02 3.77272499e-01 8.60356862e-02
 4.09478191e-01 8.02784578e-03 0.00000000e+00 6.17949392e-04
 2.68619990e-03 2.14873607e-03 2.17864128e-04 1.18258505e-03
 1.95678695e-04 4.41916328e-03]


In [40]:
predictors = ['discount_rate', 'discount_man', 'discount_jian', 'Distance']

# 建立模型 (使用 20 顆樹，每棵樹的最大深度為 4)
clf2 = RandomForestClassifier(n_estimators=20, max_depth=4)

# 訓練模型
clf2.fit(train[predictors], train['label'])

# 預測測試集
y_pred = clf2.predict(valid[predictors])

# 評分
auc_score = roc_auc_score(y_true=valid.label, y_score=y_pred)
acc = accuracy_score(y_true=valid.label, y_pred=y_pred)

print("Validation AUC: {:.3f}, Accuracy: {:.3f}".format(auc_score, acc))

# 比較重要的 feature
print(len(predictors), predictors)
print("Feature importance: ", len(clf2.feature_importances_), clf2.feature_importances_)

Validation AUC: 0.500, Accuracy: 0.952
4 ['discount_rate', 'discount_man', 'discount_jian', 'Distance']
Feature importance:  4 [0.0590689  0.32863525 0.15395856 0.45833728]


# **輸出預測檔案**

In [0]:
def exportSubmit2(model, filename):

    targetset = dftest.copy()
    print(targetset.shape)

    targetset = targetset[~targetset.Coupon_id.isna()]
    targetset.reset_index(drop=True, inplace=True)
    testset = targetset[predictors].copy()

    y_test_pred = model.predict_proba(testset[predictors])
    test1 = testset.copy()
    test1['pred_prob'] = y_test_pred[:, 1]
    print(test1.shape)
    
    output = pd.concat((targetset[["User_id", "Coupon_id", "Date_received"]], test1["pred_prob"]), axis=1)
    print(output.shape)

    output.loc[:, "User_id"] = output["User_id"].apply(lambda x:str(int(x)))
    output.loc[:, "Coupon_id"] = output["Coupon_id"].apply(lambda x:str(int(x)))
    output.loc[:, "Date_received"] = output["Date_received"].apply(lambda x:str(int(x)))
    output["uid"] = output[["User_id", "Coupon_id", "Date_received"]].apply(lambda x: '_'.join(x.values), axis=1)
    output.reset_index(drop=True, inplace=True)
    
    ### NOTE: YOUR SUBMITION FILE SHOULD HAVE COLUMN NAME: uid, label
    out = output.groupby("uid", as_index=False).mean()
    out = out[["uid", "pred_prob"]]
    out.columns = ["uid", "label"]
    out.to_csv(filename, header=["uid", "label"], index=False) # submission format
    out.head()

In [42]:
# 輸出
exportSubmit2(clf2, "Day_051_HW2_rfc1.csv")

(306313, 19)
(306313, 5)
(306313, 4)


https://github.com/wepe/O2O-Coupon-Usage-Forecast/blob/master/code/wepon/season%20one/extract_feature.py

In [43]:
#print(len(original_feature),original_feature)
print(original_feature)
#predictors = original_feature
print(predictors)

['discount_rate', 'discount_type', 'discount_man', 'discount_jian', 'Distance', 'weekday', 'weekday_type', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']
['discount_rate', 'discount_man', 'discount_jian', 'Distance']


# **增加額外的 feature**

In [0]:
#redefine processData()
def processData(df):
    
    # convert discunt_rate
    df['discount_rateNum'] = df['discount_rate'].astype('str').apply(convertRate)
    df['discount_man'] = df['discount_rate'].astype('str').apply(getDiscountMan)
    df['discount_jian'] = df['discount_rate'].astype('str').apply(getDiscountJian)
    df['discount_type'] = df['discount_rate'].astype('str').apply(getDiscountType)
    
    # convert distance
    df.loc[df.Distance.isna(), "Distance"] = 99
    return df

dfoff = processData(dfoff)
dftest = processData(dftest)

In [98]:
print(dfoff.shape)
dfoff.head()

(1160742, 22)


,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label,weekday,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,discount_rate,discount_man,discount_jian,discount_type,discount_rateNum
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0,-1,NaN,0,0,0,0,0,0,0,0,NaN,0,0,0,NaN
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,0,3.0,0,0,0,1,0,0,0,0,0.95,0,0,0,0.95
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN,0,6.0,0,0,0,0,0,0,1,0,0.95,0,0,0,0.95
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN,0,5.0,0,0,0,0,0,1,0,0,0.90,0,0,0,0.90
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN,0,5.0,0,0,0,0,0,1,0,0,0.90,0,0,0,0.90


In [102]:
dfoff=dfoff.drop("discount_rate", axis = 1)
print(dfoff.shape)

(1160742, 21)


In [103]:
dfoff.columns

Index(['User_id', 'Merchant_id', 'Coupon_id', 'Discount_rate', 'Distance',
       'Date_received', 'Date', 'label', 'weekday', 'weekday_type',
       'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5',
       'weekday_6', 'weekday_7', 'discount_man', 'discount_jian',
       'discount_type', 'discount_rateNum'],
      dtype='object')

In [99]:
dfoff.columns

Index(['User_id', 'Merchant_id', 'Coupon_id', 'Discount_rate', 'Distance',
       'Date_received', 'Date', 'label', 'weekday', 'weekday_type',
       'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5',
       'weekday_6', 'weekday_7', 'discount_rate', 'discount_man',
       'discount_jian', 'discount_type', 'discount_rateNum'],
      dtype='object')

In [94]:
dfoff.columns

Index(['User_id', 'Merchant_id', 'Coupon_id', 'Discount_rate', 'Distance',
       'Date_received', 'Date', 'label', 'weekday', 'weekday_type',
       'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5',
       'weekday_6', 'weekday_7', 'discount_rate', 'discount_man',
       'discount_jian', 'discount_type'],
      dtype='object')

In [50]:
dfoff.shape

(1160742, 21)

In [0]:
off_train = dfoff.copy()
off_test = dftest.copy()

In [0]:
def extractFeature(dataset3):
    t = dataset3[['User_id']]
    t['this_month_user_receive_all_coupon_count'] = 1
    t = t.groupby('User_id').agg('sum').reset_index()

    t1 = dataset3[['User_id','Coupon_id']]
    t1['this_month_user_receive_same_coupon_count'] = 1
    t1 = t1.groupby(['User_id','Coupon_id']).agg('sum').reset_index()

    t2 = dataset3[['User_id','Coupon_id','Date_received']]
    t2.Date_received = t2.Date_received.astype('str')
    t2 = t2.groupby(['User_id','Coupon_id'])['Date_received'].agg(lambda x:':'.join(x)).reset_index()
    t2['receive_number'] = t2.Date_received.apply(lambda s:len(s.split(':')))
    t2 = t2[t2.receive_number>1]
    t2['max_Date_received'] = t2.Date_received.apply(lambda s:max([float(d) for d in s.split(':')]))
    t2['min_Date_received'] = t2.Date_received.apply(lambda s:min([float(d) for d in s.split(':')]))
    t2 = t2[['User_id','Coupon_id','max_Date_received','min_Date_received']]

    t3 = dataset3[['User_id','Coupon_id','Date_received']]
    t3 = pd.merge(t3,t2,on=['User_id','Coupon_id'],how='left')
    t3['this_month_user_receive_same_coupon_lastone'] = t3.max_Date_received - t3.Date_received
    t3['this_month_user_receive_same_coupon_firstone'] = t3.Date_received - t3.min_Date_received
    def is_firstlastone(x):
        if x==0:
            return 1
        elif x>0:
            return 0
        else:
            return -1 #those only receive once

    t3.this_month_user_receive_same_coupon_lastone = t3.this_month_user_receive_same_coupon_lastone.apply(is_firstlastone)
    t3.this_month_user_receive_same_coupon_firstone = t3.this_month_user_receive_same_coupon_firstone.apply(is_firstlastone)
    t3 = t3[['User_id','Coupon_id','Date_received','this_month_user_receive_same_coupon_lastone','this_month_user_receive_same_coupon_firstone']]

    t4 = dataset3[['User_id','Date_received']]
    t4['this_day_user_receive_all_coupon_count'] = 1
    t4 = t4.groupby(['User_id','Date_received']).agg('sum').reset_index()

    t5 = dataset3[['User_id','Coupon_id','Date_received']]
    t5['this_day_user_receive_same_coupon_count'] = 1
    t5 = t5.groupby(['User_id','Coupon_id','Date_received']).agg('sum').reset_index()

    t6 = dataset3[['User_id','Coupon_id','Date_received']]
    t6.Date_received = t6.Date_received.astype('str')
    t6 = t6.groupby(['User_id','Coupon_id'])['Date_received'].agg(lambda x:':'.join(x)).reset_index()
    t6.rename(columns={'Date_received':'dates'},inplace=True)

    def get_day_gap_before(s):
        if str(s)=="nan":
            return -1
        Date_received,dates = s.split('-')
        dates = dates.split(':')
        gaps = []
        for d in dates:
            this_gap = (date(int(Date_received[0:4]),int(Date_received[4:6]),int(Date_received[6:8]))-date(int(d[0:4]),int(d[4:6]),int(d[6:8]))).days
            if this_gap>0:
                gaps.append(this_gap)
        if len(gaps)==0:
            return -1
        else:
            return min(gaps)

    def get_day_gap_after(s):
        if str(s)=="nan":
            return -1
        Date_received,dates = s.split('-')
        dates = dates.split(':')
        gaps = []
        for d in dates:
            this_gap = (date(int(d[0:4]),int(d[4:6]),int(d[6:8]))-date(int(Date_received[0:4]),int(Date_received[4:6]),int(Date_received[6:8]))).days
            if this_gap>0:
                gaps.append(this_gap)
        if len(gaps)==0:
            return -1
        else:
            return min(gaps)


    t7 = dataset3[['User_id','Coupon_id','Date_received']]
    t7 = pd.merge(t7,t6,on=['User_id','Coupon_id'],how='left')
    t7['Date_received_date'] = t7.Date_received.astype('str') + '-' + t7.dates
    t7['day_gap_before'] = t7.Date_received_date.apply(get_day_gap_before)
    t7['day_gap_after'] = t7.Date_received_date.apply(get_day_gap_after)
    t7 = t7[['User_id','Coupon_id','Date_received','day_gap_before','day_gap_after']]

    other_feature3 = pd.merge(t1,t,on='User_id')
    other_feature3 = pd.merge(other_feature3,t3,on=['User_id','Coupon_id'])
    other_feature3 = pd.merge(other_feature3,t4,on=['User_id','Date_received'])
    other_feature3 = pd.merge(other_feature3,t5,on=['User_id','Coupon_id','Date_received'])
    other_feature3 = pd.merge(other_feature3,t7,on=['User_id','Coupon_id','Date_received'])
    #other_feature3.to_csv('data/other_feature3.csv',index=None)
    
    return other_feature3

In [106]:
off_train_otherFeatures = extractFeature(off_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [107]:
off_train_otherFeatures.head()

,User_id,Coupon_id,this_month_user_receive_same_coupon_count,this_month_user_receive_all_coupon_count,Date_received,this_month_user_receive_same_coupon_lastone,this_month_user_receive_same_coupon_firstone,this_day_user_receive_all_coupon_count,this_day_user_receive_same_coupon_count,day_gap_before,day_gap_after
0,4,8735.0,1,1,20160214.0,-1,-1,1,1,-1,-1
1,35,1807.0,1,4,20160130.0,-1,-1,2,1,-1,-1
2,35,11951.0,2,4,20160130.0,1,0,2,1,1,-1
3,35,9776.0,1,4,20160129.0,-1,-1,2,1,-1,-1
4,35,11951.0,2,4,20160129.0,0,1,2,1,-1,1


In [108]:
off_test_otherFeatures = extractFeature(off_test)
off_test_otherFeatures.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

,User_id,Coupon_id,this_month_user_receive_same_coupon_count,this_month_user_receive_all_coupon_count,Date_received,this_month_user_receive_same_coupon_lastone,this_month_user_receive_same_coupon_firstone,this_day_user_receive_all_coupon_count,this_day_user_receive_same_coupon_count,day_gap_before,day_gap_after
0,4,2902.0,1,1,20160607.0,-1,-1,1,1,-1,-1
1,165,7571.0,1,1,20160525.0,-1,-1,1,1,-1,-1
2,166,9261.0,1,1,20160525.0,-1,-1,1,1,-1,-1
3,215,8944.0,1,1,20160524.0,-1,-1,1,1,-1,-1
4,236,11002.0,1,1,20160528.0,-1,-1,1,1,-1,-1


# **將額外的 feature 再併回去**

In [109]:
dfoff2 = pd.merge(dfoff.copy()
                  , off_train_otherFeatures           
                  ,on=['User_id','Coupon_id','Date_received']
                  ,how='left')
dfoff2.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label,weekday,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,discount_man,discount_jian,discount_type,discount_rateNum,this_month_user_receive_same_coupon_count,this_month_user_receive_all_coupon_count,this_month_user_receive_same_coupon_lastone,this_month_user_receive_same_coupon_firstone,this_day_user_receive_all_coupon_count,this_day_user_receive_same_coupon_count,day_gap_before,day_gap_after
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0,-1,NaN,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,0,3.0,0,0,0,1,0,0,0,0,0,0,0,0.95,1.0,3.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN,0,6.0,0,0,0,0,0,0,1,0,0,0,0,0.95,1.0,3.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN,0,5.0,0,0,0,0,0,1,0,0,0,0,0,0.90,1.0,1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN,0,5.0,0,0,0,0,0,1,0,0,0,0,0,0.90,1.0,1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0


In [110]:
dftest2 = pd.merge(dftest.copy()
                  , off_test_otherFeatures
                  ,on=['User_id','Coupon_id','Date_received']
                  ,how='left')
dftest2.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,weekday,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,discount_rate,discount_man,discount_jian,discount_type,discount_rateNum,this_month_user_receive_same_coupon_count,this_month_user_receive_all_coupon_count,this_month_user_receive_same_coupon_lastone,this_month_user_receive_same_coupon_firstone,this_day_user_receive_all_coupon_count,this_day_user_receive_same_coupon_count,day_gap_before,day_gap_after
0,1439408,4663,11002.0,150:20,1.0,20160528.0,6,0,0,0,0,0,0,1,0,0.866667,0,0,0,0.866667,1,3,-1,-1,1,1,-1,-1
1,1439408,2632,8591.0,20:1,0.0,20160613.0,1,0,1,0,0,0,0,0,0,0.950000,0,0,0,0.950000,2,3,1,0,1,1,28,-1
2,1439408,2632,8591.0,20:1,0.0,20160516.0,1,0,1,0,0,0,0,0,0,0.950000,0,0,0,0.950000,2,3,0,1,1,1,-1,28
3,2029232,450,1532.0,30:5,0.0,20160530.0,1,0,1,0,0,0,0,0,0,0.833333,0,0,0,0.833333,1,2,-1,-1,1,1,-1,-1
4,2029232,6459,12737.0,20:1,0.0,20160519.0,4,0,0,0,0,1,0,0,0,0.950000,0,0,0,0.950000,1,2,-1,-1,1,1,-1,-1


# **切資料**

In [111]:
dfoff2.columns

Index(['User_id', 'Merchant_id', 'Coupon_id', 'Discount_rate', 'Distance',
       'Date_received', 'Date', 'label', 'weekday', 'weekday_type',
       'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5',
       'weekday_6', 'weekday_7', 'discount_man', 'discount_jian',
       'discount_type', 'discount_rateNum',
       'this_month_user_receive_same_coupon_count',
       'this_month_user_receive_all_coupon_count',
       'this_month_user_receive_same_coupon_lastone',
       'this_month_user_receive_same_coupon_firstone',
       'this_day_user_receive_all_coupon_count',
       'this_day_user_receive_same_coupon_count', 'day_gap_before',
       'day_gap_after'],
      dtype='object')

In [112]:
dftest2.columns

Index(['User_id', 'Merchant_id', 'Coupon_id', 'Discount_rate', 'Distance',
       'Date_received', 'weekday', 'weekday_type', 'weekday_1', 'weekday_2',
       'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7',
       'discount_rate', 'discount_man', 'discount_jian', 'discount_type',
       'discount_rateNum', 'this_month_user_receive_same_coupon_count',
       'this_month_user_receive_all_coupon_count',
       'this_month_user_receive_same_coupon_lastone',
       'this_month_user_receive_same_coupon_firstone',
       'this_day_user_receive_all_coupon_count',
       'this_day_user_receive_same_coupon_count', 'day_gap_before',
       'day_gap_after'],
      dtype='object')

In [0]:
## Naive model
def split_train_valid(row, date_cut="20160416"):
    is_train = True if pd.to_datetime(row, format="%Y%m%d") < pd.to_datetime(date_cut, format="%Y%m%d") else False
    return is_train

In [114]:
df = dfoff2[dfoff2['label'] != -1].copy()
df.columns

Index(['User_id', 'Merchant_id', 'Coupon_id', 'Discount_rate', 'Distance',
       'Date_received', 'Date', 'label', 'weekday', 'weekday_type',
       'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5',
       'weekday_6', 'weekday_7', 'discount_man', 'discount_jian',
       'discount_type', 'discount_rateNum',
       'this_month_user_receive_same_coupon_count',
       'this_month_user_receive_all_coupon_count',
       'this_month_user_receive_same_coupon_lastone',
       'this_month_user_receive_same_coupon_firstone',
       'this_day_user_receive_all_coupon_count',
       'this_day_user_receive_same_coupon_count', 'day_gap_before',
       'day_gap_after'],
      dtype='object')

In [115]:
#df = dfoff2[dfoff2['label'] != -1].copy()
df["is_train"] = df["Date_received"].apply(split_train_valid)

train = df[df["is_train"]]
valid = df[~df["is_train"]]

train.reset_index(drop=True, inplace=True)
valid.reset_index(drop=True, inplace=True)

print("Train size: {}, #positive: {}".format(len(train), train["label"].sum()))
print("Valid size: {}, #positive: {}".format(len(valid), valid["label"].sum()))

Train size: 952393, #positive: 84886
Valid size: 83692, #positive: 6382


In [116]:
train.shape

(952393, 30)

In [73]:
train.shape

(952393, 30)

In [118]:
train.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label,weekday,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,discount_man,discount_jian,discount_type,discount_rateNum,this_month_user_receive_same_coupon_count,this_month_user_receive_all_coupon_count,this_month_user_receive_same_coupon_lastone,this_month_user_receive_same_coupon_firstone,this_day_user_receive_all_coupon_count,this_day_user_receive_same_coupon_count,day_gap_before,day_gap_after,is_train
0,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,0,3.0,0,0,0,1,0,0,0,0,0,0,0,0.95,1.0,3.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,True
1,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN,0,6.0,0,0,0,0,0,0,1,0,0,0,0,0.95,1.0,3.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,True
2,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN,0,5.0,0,0,0,0,0,1,0,0,0,0,0,0.90,1.0,1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,True
3,2223968,3381,9776.0,10:5,2.0,20160129.0,NaN,0,5.0,0,0,0,0,0,1,0,0,0,0,0,0.50,1.0,1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,True
4,73611,2099,12034.0,100:10,99.0,20160207.0,NaN,0,7.0,0,0,0,0,0,0,0,1,0,0,0,0.90,1.0,1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,True


In [124]:
original_feature = ['discount_rateNum',
                    'discount_type',
                    'discount_man', 
                    'discount_jian',
                    'Distance', 
                    'weekday', 
                    'weekday_type'] + weekdaycols

print(original_feature)

['discount_rateNum', 'discount_type', 'discount_man', 'discount_jian', 'Distance', 'weekday', 'weekday_type', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']


In [125]:
original_feature = original_feature + ['Date_received'
                                       , 'this_month_user_receive_same_coupon_count'
                                       , 'this_month_user_receive_all_coupon_count'
                                       , 'this_month_user_receive_same_coupon_lastone'
                                       , 'this_month_user_receive_same_coupon_firstone'
                                       , 'this_day_user_receive_all_coupon_count'
                                       , 'this_day_user_receive_same_coupon_count'
                                       , 'day_gap_before'
                                       , 'day_gap_after']

print(len(original_feature),original_feature)

23 ['discount_rateNum', 'discount_type', 'discount_man', 'discount_jian', 'Distance', 'weekday', 'weekday_type', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7', 'Date_received', 'this_month_user_receive_same_coupon_count', 'this_month_user_receive_all_coupon_count', 'this_month_user_receive_same_coupon_lastone', 'this_month_user_receive_same_coupon_firstone', 'this_day_user_receive_all_coupon_count', 'this_day_user_receive_same_coupon_count', 'day_gap_before', 'day_gap_after']


# **模型 SGDClassifier**

In [126]:
predictors = original_feature
print(predictors)

['discount_rateNum', 'discount_type', 'discount_man', 'discount_jian', 'Distance', 'weekday', 'weekday_type', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7', 'Date_received', 'this_month_user_receive_same_coupon_count', 'this_month_user_receive_all_coupon_count', 'this_month_user_receive_same_coupon_lastone', 'this_month_user_receive_same_coupon_firstone', 'this_day_user_receive_all_coupon_count', 'this_day_user_receive_same_coupon_count', 'day_gap_before', 'day_gap_after']


In [0]:
def check_model(data, predictors):
    
    classifier = lambda: SGDClassifier(
        loss='log', 
        penalty='elasticnet', 
        fit_intercept=True, 
        max_iter=100, 
        shuffle=True, 
        n_jobs=1,
        class_weight=None)

    model = Pipeline(steps=[
        ('ss', StandardScaler()),
        ('en', classifier())
    ])

    parameters = {
        'en__alpha': [ 0.001, 0.01, 0.1],
        'en__l1_ratio': [ 0.001, 0.01, 0.1]
    }

    folder = StratifiedKFold(n_splits=3, shuffle=True)
    
    grid_search = GridSearchCV(
        model, 
        parameters, 
        cv=folder, 
        n_jobs=-1, 
        verbose=1)
    grid_search = grid_search.fit(data[predictors], 
                                  data['label'])
    
    return grid_search

In [128]:
model = check_model(train, predictors)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:  1.0min finished


In [0]:
y_valid_pred = model.predict_proba(valid[predictors])
valid1 = valid.copy()
valid1['pred_prob'] = y_valid_pred[:, 1]

In [130]:
from sklearn.metrics import roc_auc_score, accuracy_score
auc_score = roc_auc_score(y_true=valid.label, y_score=y_valid_pred[:,1])
acc = accuracy_score(y_true=valid.label, y_pred=y_valid_pred.argmax(axis=1))
print("Validation AUC: {:.3f}, Accuracy: {:.3f}".format(auc_score, acc))

Validation AUC: 0.773, Accuracy: 0.940


In [133]:
targetset = dftest2.copy()
print(targetset.shape)
targetset = targetset[~targetset.Coupon_id.isna()]
targetset.reset_index(drop=True, inplace=True)
testset = targetset[predictors].copy()

y_test_pred = model.predict_proba(testset[predictors])
test1 = testset.copy()
test1['pred_prob'] = y_test_pred[:, 1]
print(test1.shape)

(325057, 28)
(325057, 24)


In [137]:
output = pd.concat((targetset[["User_id", "Coupon_id", "Date_received"]], test1["pred_prob"]), axis=1)
print(output.shape)

output.loc[:, "User_id"] = output["User_id"].apply(lambda x:str(int(x)))
output.loc[:, "Coupon_id"] = output["Coupon_id"].apply(lambda x:str(int(x)))
output.loc[:, "Date_received"] = output["Date_received"].apply(lambda x:str(int(x)))
output["uid"] = output[["User_id", "Coupon_id", "Date_received"]].apply(lambda x: '_'.join(x.values), axis=1)
output.reset_index(drop=True, inplace=True)

(325057, 4)


In [138]:
### NOTE: YOUR SUBMITION FILE SHOULD HAVE COLUMN NAME: uid, label
out = output.groupby("uid", as_index=False).mean()
out = out[["uid", "pred_prob"]]
out.columns = ["uid", "label"]
out.head()

,uid,label
0,1000020_2705_20160519,0.133501
1,1000020_8192_20160513,0.116594
2,1000065_1455_20160527,0.058681
3,1000085_8067_20160513,0.090258
4,1000086_2418_20160613,0.126959


In [0]:
out.to_csv("Day_051_HW_extractfeatur1.csv", header=["uid", "label"], index=False) # submission format
#0.66613

# **輸出預測檔案**

In [0]:
def exportSubmit(model, filename):

    targetset = dftest2.copy()
    print(targetset.shape)

    targetset = targetset[~targetset.Coupon_id.isna()]
    targetset.reset_index(drop=True, inplace=True)
    testset = targetset[predictors].copy()

    y_test_pred = model.predict_proba(testset[predictors])
    test1 = testset.copy()
    test1['pred_prob'] = y_test_pred[:, 1]
    print(test1.shape)
    
    output = pd.concat((targetset[["User_id", "Coupon_id", "Date_received"]], test1["pred_prob"]), axis=1)
    print(output.shape)

    output.loc[:, "User_id"] = output["User_id"].apply(lambda x:str(int(x)))
    output.loc[:, "Coupon_id"] = output["Coupon_id"].apply(lambda x:str(int(x)))
    output.loc[:, "Date_received"] = output["Date_received"].apply(lambda x:str(int(x)))
    output["uid"] = output[["User_id", "Coupon_id", "Date_received"]].apply(lambda x: '_'.join(x.values), axis=1)
    output.reset_index(drop=True, inplace=True)
    
    ### NOTE: YOUR SUBMITION FILE SHOULD HAVE COLUMN NAME: uid, label
    out = output.groupby("uid", as_index=False).mean()
    out = out[["uid", "pred_prob"]]
    out.columns = ["uid", "label"]
    out.to_csv(filename, header=["uid", "label"], index=False) # submission format
    out.head()

# **模型 RandomForestClassifier**

In [147]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score

# 建立模型 (使用 20 顆樹，每棵樹的最大深度為 4)
model_rfc = RandomForestClassifier(n_estimators=20, max_depth=4)

# 訓練模型
model_rfc.fit(train[predictors], train['label'])

# 預測測試集
pred = model_rfc.predict(valid[predictors])

# 評分
auc_score = roc_auc_score(y_true=valid.label, y_score=pred)
acc = accuracy_score(y_true=valid.label, y_pred=pred)

print("Validation AUC: {:.3f}, Accuracy: {:.3f}".format(auc_score, acc))

Validation AUC: 0.666, Accuracy: 0.945


In [148]:
# 輸出
exportSubmit(model_rfc, "Day_051_HW_extractfeature_rfc1.csv")
#0.75317

(325057, 28)
(325057, 24)
(325057, 4)
